In [40]:
import psycopg2
import numpy as np
import pandas as pd

In [41]:
column_names = ["level_0",
  "index",
  "pitch_type",
  "game_date",
  "release_speed",
  "release_pos_x",
  "release_pos_z",
  "player_name",
  "batter",
  "pitcher",
  "events",
  "description",
  "zone",
  "stand",
  "p_throws",
  "home_team",
  "away_team",
  "type",
  "hit_location",
  "bb_type",
  "balls",
  "strikes",
  "game_year",
  "pfx_x",
  "pfx_z",
  "plate_x",
  "plate_z",
  "on_3b",
  "on_2b",
  "on_1b",
  "outs_when_up",
  "inning",
  "inning_topbot",
  "hc_x",
  "hc_y",
  "fielder_2",
  "sv_id",
  "vx0",
  "vy0",
  "vz0",
  "ax",
  "ay",
  "az",
  "sz_top",
  "sz_bot",
  "hit_distance_sc",
  "launch_speed",
  "launch_angle",
  "effective_speed",
  "release_spin_rate",
  "release_extension",
  "game_pk",
  "pitcher.1",
  "fielder_2.1",
  "fielder_3",
  "fielder_4",
  "fielder_5",
  "fielder_6",
  "fielder_7",
  "fielder_8",
  "fielder_9",
  "release_pos_y",
  "estimated_ba_using_speedangle",
  "estimated_woba_using_speedangle",
  "woba_value",
  "woba_denom",
  "babip_value",
  "iso_value",
  "launch_speed_angle",
  "at_bat_number",
  "pitch_number",
  "pitch_name",
  "home_score",
  "away_score",
  "bat_score",
  "fld_score",
  "post_away_score",
  "post_home_score",
  "post_bat_score",
  "post_fld_score",
  "if_fielding_alignment",
  "of_fielding_alignment",
  "pitch_subtype",
  "count_status",
  "score_differential",
  "bases_loaded",
  "batter_swung",
  "ball_high",
  "ball_low",
  "ball_left",
  "ball_right",
  "in_strikezone",
  "chased"]

In [42]:
con = psycopg2.connect(dbname= 'dev', host='examplecluster.cdbpwaymevt5.us-east-2.redshift.amazonaws.com',
                       port= '5439', user= 'awsuser', password= '47Westrange')

cur = con.cursor()

In [43]:
cur.execute("""SELECT * FROM pitches""")

KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(cur.fetchall(), columns = column_names)

In [ ]:
df.head()

In [ ]:
cur.close() 
con.close()

In [ ]:
def make_batters_df(prior_df):
    df = prior_df.copy()
    #make list of the unique batter ids
    batters = list(df['batter'].unique())
    #initialize empty dictionary to store the batter stats
    batters_dict = {}
    #set a break flag to False for error-checking
    brk = False
    #iterate thru each unique batter
    for batter in batters:
        if brk:
            break
        #make subset of the df for that batter and assign to variable batter_df    
        batter_df = df[df['batter'] == batter]
        
        #assign all pitch categories to list:
        all_pitch_cats = ['fastball', 'breaking', 'offspeed', 'pitchout']
        
        #assign the pitch categories to a list
        try:
            pitch_cats = batter_df['pitch_type'].unique().tolist()
        except KeyError:
            print(batter)
            brk = True
        #get the normalized value counts of pitches by category that batter has faced
        vc = batter_df['pitch_type'].value_counts(normalize=True)
        #initialize empty dict for each batter
        batter_dict = {}
        
        #if there are any pitch categories the batter has not faced, 
        unfaced_cats = list(set(all_pitch_cats) - set(pitch_cats))
        
        for cat in pitch_cats:
            if brk:
                break
        
            #assign the % of pitches faced by the batter for that category to his batter dict
            try:
                batter_dict[cat + '_perc_faced'] = vc[cat] * 100
            except TypeError:
                print(batter)
                return 1
                
            #continue out of the loop for pitchout category since ball in play stats are NaN
            if cat == 'pitchout':
                continue
        
            #grab subset of batter df for the pitch category
            cat_df = batter_df[batter_df['pitch_type'] == cat]
            
            #if he has faced less than 100 pitches of that type, add it to unfaced_category and fill w NaN
            if len(cat_df) < 100:
                unfaced_cats.append(cat)
                continue
        
            #calculate batters chase % for pitch type category on balls outside the strikezone
            out_of_strikezone = len(cat_df) - cat_df['in_strikezone'].sum() #num of times ball was out of zone
            chased_count = cat_df['chased'].sum() #num of times batter chased
            try:
                chase_perc = (chased_count / out_of_strikezone) * 100
            except ZeroDivisionError:
                chase_perc = np.nan
            #assign the chase perc to the batter dict
            batter_dict[cat + '_chase_perc'] = chase_perc
        
            #calc ball in play % for each swing for each pitch cat:
            ball_in_play_count = len(cat_df[cat_df['type'] == 'X']) #type X means ball hit into play
            swung_count = cat_df['batter_swung'].sum() #counts all the 1s in the swung column
            #assign the ball in play % per swing to the batter dict
            batter_dict[cat + '_bip_swung_perc'] = (ball_in_play_count / swung_count) * 100
        
            #calculate taken strike %
            taken_strike_count = len(cat_df[(cat_df['in_strikezone'] == 1) & (cat_df['batter_swung'] == 0)])
            pitches_in_zone_count = cat_df['in_strikezone'].sum() #counts the 1s in the in zone col
            #assign to batter_dict
            batter_dict[cat + '_taken_strike_perc'] = (taken_strike_count / pitches_in_zone_count) * 100
        
            #for each pitch type category, get the batters stats on balls hit in play
            stats = ['estimated_woba_using_speedangle', 'babip_value', 'iso_value']
            for stat in stats:
                #drop Nans from the stat column and assign to new subset, for each stat
                stat_cat_df = cat_df.dropna(subset=[stat])
                if stat == 'estimated_woba_using_speedangle':
                    #get the mean avg_est_woba
                    avg_est_woba = stat_cat_df['estimated_woba_using_speedangle'].mean()
                    #assign that value to the batters dictionary
                    batter_dict[cat + '_est_woba'] = avg_est_woba
                    if avg_est_woba == np.nan:
                        print(batter)
                        brk = True
                        break
                elif stat == 'babip_value':
                    avg_babip = stat_cat_df['babip_value'].mean()
                    batter_dict[cat + '_babip'] = avg_babip
                else:
                    avg_iso_value = stat_cat_df['iso_value'].mean()
                    batter_dict[cat + '_iso_value'] = avg_iso_value
                    
        #for unfaced or small sample pitch_types: assign NaNs to his dictionary for that category
        for cat in unfaced_cats:
            if cat == 'pitchout':
                batter_dict[cat + '_perc_faced'] = 0
            else:
                batter_dict[cat + '_perc_faced'] = np.nan
                batter_dict[cat + '_chase_perc'] = np.nan
                batter_dict[cat + '_bip_swung_perc'] = np.nan
                batter_dict[cat + '_taken_strike_perc'] = np.nan
                batter_dict[cat + '_est_woba'] = np.nan
                batter_dict[cat + '_babip'] = np.nan
                batter_dict[cat + '_iso_value'] = np.nan
        
        #assign the batter dictionary to the main dictionary of all batters
        batters_dict[batter] = batter_dict
    if not brk:
        print('iteration completed successfully')
        
    #make df from the batters dict
    batters_df = pd.DataFrame.from_dict(batters_dict, orient='index')
    batters_df = batters_df.reset_index().rename(columns={'index':'batter'})
    return batters_df

In [ ]:
batters_df = make_batters_df(df)
batters_df.head()

In [ ]:
batter_cols = ['fastball_perc_faced','fastball_chase_perc','fastball_bip_swung_perc', 'fastball_taken_strike_perc',
               'fastball_est_woba', 'fastball_babip', 'fastball_iso_value', 'breaking_perc_faced', 'breaking_chase_perc',
               'breaking_bip_swung_perc', 'breaking_taken_strike_perc', 'breaking_est_woba', 'breaking_babip', 
               'breaking_iso_value', 'offspeed_perc_faced', 'offspeed_chase_perc', 'offspeed_bip_swung_perc',
               'offspeed_taken_strike_perc', 'offspeed_est_woba', 'offspeed_babip', 'offspeed_iso_value',
               'pitchout_perc_faced']

def fill_batting_nans(pitcher_df, batting_order_slot_map):
    df = pitcher_df.copy()
    for slot in df['batting_order_slot'].unique().tolist():
        subset = df[df['batting_order_slot'] == slot].copy()
        df = df.drop(subset.index)
        for col in batter_cols:
            subset[col] = subset[col].fillna(batting_order_slot_map[slot][col])
        df = pd.concat([df, subset])
        print('finished w/ slot: ' + str(slot))
    df = df.sort_values(by=['game_date', 'game_pk', 'pitch_count'])
    return df